<a href="https://www.kaggle.com/code/hatemamine/scotuslexglue-preprocesslongtextwithkeybert?scriptVersionId=132282892" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
!pip install keybert --quiet
from keybert import KeyBERT
from datasets import load_dataset
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import pandas 
import numpy as np 
import re
import matplotlib
import seaborn as sb 
import time

In [ ]:
#get the number of words 
def getNumberOfWords(df):
    maxl=[]
    for i in range(len(df)):
        x= df["ATexte"][i].split()
        if len(x)< 3: print(i)
        maxl.append(len(x))
    print("Max value element : ", max(maxl))
    print("Min value element : ", min(maxl))
    return maxl
    

In [ ]:
def plot_NumberOfWord(maxl, dataset_name):
    fig = plt.figure()
    ax1 = plt.axes()
    ax1.set_ylabel('words count')
    ax1.set_xlabel('instances')
    ind = np.arange(len(maxl))
    line, =ax1.plot(ind,maxl)
    plt.axhline(y=512, color="red", linestyle="--")
    fig.savefig(dataset_name+"words count.png")
    plt.show()

In [ ]:
def plotNumberSampleforeachCLASS(df, dataset_name):
    sb.countplot(df.MotsCles)
    matplotlib.pyplot.ylabel("Number of samples per Class")
    matplotlib.pyplot.xlabel("Class label")
    matplotlib.pyplot.savefig(dataset_name+'distribution.png')
    plt.show()

In [ ]:
def PreprocessLongTextwithKeyBERT(df, n_gram):
    kw_model = KeyBERT()
    start_time = time.time()
    for i in tqdm(range(len(df))):
        tmpnewtext=""
        x= df["ATexte"][i].split()
        #if len(x) < 513 : continue 
        textgen=""
        for j in range(len(x)):
            textgen =textgen +" "+x[j]
            if j ==0:continue # for not process the first empty string
            if (j % 512 ==0) or (len(x)-1==j): 
                keygen = kw_model.extract_keywords(textgen, keyphrase_ngram_range=(n_gram, n_gram), stop_words='english', use_mmr=True, diversity=0.9, top_n=int(max(1, round((512*512)/(len(x)*1), 0))))
                textgen=""
                for y in range(len(keygen)):
                    tmpnewtext=tmpnewtext +" "+keygen[y][0]
                #print(keygen[y][0])
        df["ATexte"][i]=tmpnewtext
        #break
    print("--- %s seconds ---" % (time.time() - start_time))
    return df

In [ ]:
def plotNumberSampleforeachCLASS(df, dataset_name):
    plot_=sb.countplot(df.MotsCles)
    #new_ticks = [i.get_text() for i in plot_.get_xticklabels()]
    #matplotlib.pyplot.xticks(range(0, len(new_ticks), 10), new_ticks[::10], rotation='vertical')
    matplotlib.pyplot.ylabel("Number of samples per Class")
    matplotlib.pyplot.xlabel("Class label")
    matplotlib.pyplot.savefig(dataset_name+'distribution.png')
    plt.show()

In [ ]:
# To ignore warinings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
dataset = load_dataset("lex_glue", "scotus")
DATASET_NAME='SCOTUS'
for x in dataset.keys():
    #get the train dataset
    dict = {'ATexte': dataset[x]["text"], 'MotsCles': dataset[x]["label"]}
    df = pandas.DataFrame(dict)
    df["ATexte"]= df["ATexte"].str.lower()
    for i in range(len(df)):
        df["ATexte"][i] = re.sub(r'[^a-z ]', '', df["ATexte"][i])
    print(x,len(df),"sample")
    plotNumberSampleforeachCLASS(df, DATASET_NAME+x)
    plot_NumberOfWord(getNumberOfWords(df), DATASET_NAME+x)
    df.to_csv("/kaggle/working/"+DATASET_NAME+x+".csv")
    df = PreprocessLongTextwithKeyBERT(df, 3)
    df = PreprocessLongTextwithKeyBERT(df, 1)
    plot_NumberOfWord(getNumberOfWords(df), DATASET_NAME+x+"KeyBERT")
    df.to_csv("/kaggle/working/"+DATASET_NAME+x+"KeyBERT.csv")

In [ ]:
import pandas 
import numpy as np 
import matplotlib.pyplot as plt
import cv2
#get the train dataset
df = pandas.read_csv("/kaggle/working/SCOTUStrainKeyBERT.csv")
#get the test dataset to augment the training data and live the validation data for evaluation
df_test = pandas.read_csv("/kaggle/working/SCOTUStestKeyBERT.csv")
print(len(df))
print(df.head())

In [ ]:
import random
random.seed(1)
text1=[]
text2=[]
correlated=[]
labels=[]

for i in range (len(df)):
    row0 =df.iloc[i]
    # we exclud label 0 and 1 from the generated dataset for training 
    if row0["MotsCles"]==0 or row0["MotsCles"]==1:continue
    x=0
    y=0
    while x < 30:
        ind = random.randint(0, len(df)-1)
        if i ==ind : continue
        # we exclud label 0 and 1 from the generated dataset for training
        row =df.iloc[ind]
        if row0["MotsCles"]==row["MotsCles"]:
            text1.append(row0["ATexte"])
            text2.append(row["ATexte"])
            correlated.append(1)
            labels.append(str(row0["MotsCles"])+"=="+str(row["MotsCles"]))
            x=x+1
            
        
    while y < 30:
        ind = random.randint(0, len(df)-1)
        if i ==ind : continue
        row =df.iloc[ind]
        if row0["MotsCles"]!=row["MotsCles"]:
            text1.append(row0["ATexte"])
            text2.append(row["ATexte"])
            correlated.append(0)
            labels.append(str(row0["MotsCles"])+"!="+str(row["MotsCles"]))
            y=y+1
    #break
dict = {'ATexte':text1 , 'MotsCles':text2, 'correlated':correlated, 'labels':labels } 
df7 = pandas.DataFrame(dict)              
print(len(df7))
print(df7.head(2))

In [ ]:
!pip install sentence-transformers --quiet
train, validate = np.split(df7.sample(frac=1, random_state=42),[int(.9*len(df7))])
#print(len(df7))
#print(len(train))
#print(len(validate))
#print(train.head())
from sentence_transformers import InputExample
dev_samples=[]
train_samples=[]
for i in range(len(train)):
        row =train.iloc[i]
        # two input text and the target (1=similarity, 0=disimilarity)
        train_samples.append(InputExample(texts=[row["ATexte"], row["MotsCles"]], label=float(row["correlated"])))
        #break
for i in range(len(validate)):
        row =validate.iloc[i]
        dev_samples.append(InputExample(texts=[row["ATexte"], row["MotsCles"]], label=float(row["correlated"])))
        #break
#print(train_samples[:10])

In [ ]:
# save the generated dataset in a pickle file 
import pickle
import torch
lexgluescotus = {'train_samples':train_samples, 'dev_samples':dev_samples}
with open('/kaggle/working/key_lexgluescotusFSL01.pickle', 'wb') as handle:
    pickle.dump(lexgluescotus, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
df =pandas.concat([df, df_test], ignore_index=True)

In [ ]:
import random
random.seed(1)
text1=[]
text2=[]
correlated=[]
labels=[]

for i in range (len(df)):
    row0 =df.iloc[i]
    # we exclud label 0 and 1 from the generated dataset for training 
    if row0["MotsCles"]==0 or row0["MotsCles"]==1:continue
    x=0
    y=0
    while x < 40:
        ind = random.randint(0, len(df)-1)
        if i ==ind : continue
        # we exclud label 0 and 1 from the generated dataset for training
        row =df.iloc[ind]
        if row0["MotsCles"]==row["MotsCles"]:
            text1.append(row0["ATexte"])
            text2.append(row["ATexte"])
            correlated.append(1)
            labels.append(str(row0["MotsCles"])+"=="+str(row["MotsCles"]))
            x=x+1
            
        
    while y < 40:
        ind = random.randint(0, len(df)-1)
        if i ==ind : continue
        row =df.iloc[ind]
        if row0["MotsCles"]!=row["MotsCles"]:
            text1.append(row0["ATexte"])
            text2.append(row["ATexte"])
            correlated.append(0)
            labels.append(str(row0["MotsCles"])+"!="+str(row["MotsCles"]))
            y=y+1
    #break
dict = {'ATexte':text1 , 'MotsCles':text2, 'correlated':correlated, 'labels':labels } 
df7 = pandas.DataFrame(dict)              
print(len(df7))
print(df7.head(2))

In [ ]:
train, validate = np.split(df7.sample(frac=1, random_state=42),[int(.9*len(df7))])
#print(len(df7))
#print(len(train))
#print(len(validate))
#print(train.head())
from sentence_transformers import InputExample
dev_samples=[]
train_samples=[]
for i in range(len(train)):
        row =train.iloc[i]
        # two input text and the target (1=similarity, 0=disimilarity)
        train_samples.append(InputExample(texts=[row["ATexte"], row["MotsCles"]], label=float(row["correlated"])))
        #break
for i in range(len(validate)):
        row =validate.iloc[i]
        dev_samples.append(InputExample(texts=[row["ATexte"], row["MotsCles"]], label=float(row["correlated"])))
        #break
#print(train_samples[:10])

In [ ]:
# save the generated dataset in a pickle file 
import pickle
import torch
lexgluescotus = {'train_samples':train_samples, 'dev_samples':dev_samples}
with open('/kaggle/working/Augmentkey_lexgluescotusFSL01.pickle', 'wb') as handle:
    pickle.dump(lexgluescotus, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
#get the train dataset
df = pandas.read_csv("/kaggle/working/SCOTUStrain.csv")
print(len(df))
print(df.head())

In [ ]:
import random
random.seed(1)
text1=[]
text2=[]
correlated=[]
labels=[]

for i in range (len(df)):
    row0 =df.iloc[i]
    # we exclud label 0 and 1 from the generated dataset for training 
    if row0["MotsCles"]==0 or row0["MotsCles"]==1:continue
    x=0
    y=0
    while x < 30:
        ind = random.randint(0, len(df)-1)
        if i ==ind : continue
        # we exclud label 0 and 1 from the generated dataset for training
        row =df.iloc[ind]
        if row0["MotsCles"]==row["MotsCles"]:
            text1.append(row0["ATexte"])
            text2.append(row["ATexte"])
            correlated.append(1)
            labels.append(str(row0["MotsCles"])+"=="+str(row["MotsCles"]))
            x=x+1
            
        
    while y < 30:
        ind = random.randint(0, len(df)-1)
        if i ==ind : continue
        row =df.iloc[ind]
        if row0["MotsCles"]!=row["MotsCles"]:
            text1.append(row0["ATexte"])
            text2.append(row["ATexte"])
            correlated.append(0)
            labels.append(str(row0["MotsCles"])+"!="+str(row["MotsCles"]))
            y=y+1
    #break
dict = {'ATexte':text1 , 'MotsCles':text2, 'correlated':correlated, 'labels':labels } 
df7 = pandas.DataFrame(dict)              
print(len(df7))
print(df7.head(2))

In [ ]:
!pip install sentence-transformers --quiet
train, validate = np.split(df7.sample(frac=1, random_state=42),[int(.9*len(df7))])
#print(len(df7))
#print(len(train))
#print(len(validate))
#print(train.head())
from sentence_transformers import InputExample
dev_samples=[]
train_samples=[]
for i in range(len(train)):
        row =train.iloc[i]
        # two input text and the target (1=similarity, 0=disimilarity)
        train_samples.append(InputExample(texts=[row["ATexte"], row["MotsCles"]], label=float(row["correlated"])))
        #break
for i in range(len(validate)):
        row =validate.iloc[i]
        dev_samples.append(InputExample(texts=[row["ATexte"], row["MotsCles"]], label=float(row["correlated"])))
        #break
#print(train_samples[:10])

In [ ]:
# save the generated dataset in a pickle file 
import pickle
import torch
lexgluescotus = {'train_samples':train_samples, 'dev_samples':dev_samples}
with open('/kaggle/working/lexgluescotusFSL01.pickle', 'wb') as handle:
    pickle.dump(lexgluescotus, handle, protocol=pickle.HIGHEST_PROTOCOL)